## <div class="alert alert-block alert-warning">This notebook includes two folium map. It takes some seconds to load. Please wait...⌛</div>

# <div class="alert alert-block alert-success">About this notebook</div>

Thinking very simply, hospitals should grow with some factors such as population, for example. 

So it might not be strange to assume that the numbers between hospitals in public and private sectors have simple relarion.

In this notebook, I focus on the rate of hospital of sectors in each region.

I made scatter plot and thought that there maight be simple relation between number of hospitals. 

If there are simple relation, the outliers may have some meaning. I was interested in it.

## <div class="alert alert-block alert-info">Preperation</div>

## Data overview

First, I load dataset and library, and check overveiw, type and etc... 

I found that this dataset includes not numbers but strings, so I'll convert them.

In [ ]:
import json
import os

import folium
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import seaborn as sns

There are one data file.

In [ ]:
!ls ../input/hospitals-count-in-india-statewise

Column names have space, so I'll rename them. It just feels little uncomfortable.

In [ ]:
df = pd.read_csv("../input/hospitals-count-in-india-statewise/Hospitals count in India - Statewise.csv")
df = df.rename(columns={'Number of hospitals in public sector': 'Number_of_hospitals_in_public_sector',
                        'Number of hospitals in private sector': 'Number_of_hospitals_in_private_sector',
                        'Total number of hospitals (public+private)': 'Total_number_of_hospitals_public_private'},)

In [ ]:
df

In [ ]:
df.info()

This dataset includes not numbers but strings. I'll convert the strings to numbers.

In [ ]:
feature_columns = ['Number_of_hospitals_in_public_sector', 'Number_of_hospitals_in_private_sector', 'Total_number_of_hospitals_public_private']

In [ ]:
def convert_str_with_commma2int(x):
    if type(x) is str:
        return int(x.replace(',',''))
    else:
        return x

for col in feature_columns:
    df[col] = df[col].map(convert_str_with_commma2int)
    
# I recalculate total number because total number of Chhattisgarh seems wrong.
df["Total_number_of_hospitals_public_private"] = df["Number_of_hospitals_in_public_sector"] + df["Number_of_hospitals_in_private_sector"]
    
df.head()

In [ ]:
df.describe()

# <div class="alert alert-block alert-info">Visualization</div>

First, I'll see histgram of hospitals in each sector.

In [ ]:
g_public_h_hist = sns.distplot(df["Number_of_hospitals_in_public_sector"], kde=False, rug=False)
g_public_h_hist.set_title("Number_of_hospitals_in_public_sector")

In [ ]:
g_private_h_hist = sns.distplot(df["Number_of_hospitals_in_private_sector"], kde=False, rug=False)
g_private_h_hist.set_title("Number_of_hospitals_in_private_sector")

In [ ]:
g_total_h_hist = sns.distplot(df["Total_number_of_hospitals_public_private"], kde=False, rug=False)
g_total_h_hist.set_title("Total_number_of_hospitals_public_private")

Inspite of sectors, the trends seem to be same.

Next, I'll see pie chart to know which region has many hospital and has less hospital.

In [ ]:
df_sumup = df.groupby('States/UTs', as_index=False).sum()
df_sumup.head()

In [ ]:
fig = px.pie(df_sumup, values='Number_of_hospitals_in_public_sector', names='States/UTs', title='Number_of_hospitals_in_public_sector')
fig.show()

In [ ]:
fig = px.pie(df_sumup, values='Number_of_hospitals_in_private_sector', names='States/UTs', title='Number_of_hospitals_in_public_sector')
fig.show()

In [ ]:
fig = px.pie(df_sumup, values='Total_number_of_hospitals_public_private', names='States/UTs', title='Number_of_hospitals_in_public_sector')
fig.show()

# <div class="alert alert-block alert-info">EDA</div>

## Search relation by scatter plot

Next, let's explore the relationship between numbers of hospitals in public and private sectors from the scatter plot.

Indeed, as the number of hospitals in one sector increases, number of hospitals in the other is increases, too. 
It’s a bit rough, but based on the shape of the distribution, it seems to be okay to approximate it with linear regression.

In [ ]:
plt.figure(figsize=(20, 10))
g = sns.scatterplot(data=df, x="Number_of_hospitals_in_public_sector", y="Number_of_hospitals_in_private_sector")
g.set_title("Relation between number of hospitals in public sector and private sector")

In [ ]:
plt.figure(figsize=(20, 10))
g = sns.regplot(data=df, x="Number_of_hospitals_in_public_sector", y="Number_of_hospitals_in_private_sector", ci=95)
g.set_title("Relation between number of hospitals in public sector and private sector with lenear regression and 95% confidence interval")

In Himachal Pradesh 8, Assam, West Bengal, Odisha and hospitals, compared to other states, the percentage of hospitals of in public sector seems to be higher.

On the other hands, in Chhattisgarh, Maharashtra ,Telangana, Karnataka and Uttar Pradesh, the percentage of hospitals of in private sector seems to be higher.

I'll check the states.

In [ ]:
df_high_public = df[(df["Number_of_hospitals_in_public_sector"] <= 2000) &
                    ( 700 <= df["Number_of_hospitals_in_public_sector"] ) & 
                    (df["Number_of_hospitals_in_private_sector"] <= 1000)]

In [ ]:
df_high_public

In [ ]:
df_high_private = df[(df["Number_of_hospitals_in_public_sector"] <= 1000) &
                     ( 1800 <= df["Number_of_hospitals_in_private_sector"] ) & 
                     (df["Number_of_hospitals_in_private_sector"] <= 4000) | 
                     (df["Number_of_hospitals_in_private_sector"] > 6000)]

In [ ]:
df_high_private

## View from a geographic point of view

We get the information which states are outliers. So I'll check where the states are in map, and explore geographical features.

In [ ]:
#I refered https://www.kaggle.com/niharika41298/covid-19-india-analysis

with open('../input/indian-state-geojson-data/india_state_geo.json') as file:
    geojsonData = json.load(file)
        
for i in geojsonData['features']:
    i['id'] = i['properties']['NAME_1']

In [ ]:
#To display area, rename "Himachal Pradesh 8" and "Orissa".
df_high_public["States/UTs"][19] = 'Himachal Pradesh'
df_high_public["States/UTs"][28] = 'Orissa'

### States which rate of hospital of public sector is high.

In [ ]:
map_choropleth_high_public = folium.Map(location = [20.5937,78.9629], zoom_start = 4)

In [ ]:
folium.Choropleth(geo_data=geojsonData,
                 data=df_high_public,
                 name='CHOROPLETH',
                 key_on='feature.id',
                 columns = ['States/UTs','Number_of_hospitals_in_public_sector'],
                 fill_color='YlOrRd',
                 fill_opacity=0.7,
                 line_opacity=0.8,
                 legend_name='High rate in public sector hospital',
                 highlight=True).add_to(map_choropleth_high_public)

folium.LayerControl().add_to(map_choropleth_high_public)
display(map_choropleth_high_public)

### States which rate of hospital of private sector is high.

In [ ]:
map_choropleth_high_private = folium.Map(location = [20.5937,78.9629], zoom_start = 4)

In [ ]:
folium.Choropleth(geo_data=geojsonData,
                 data=df_high_private,
                 name='CHOROPLETH',
                 key_on='feature.id',
                 columns = ['States/UTs','Number_of_hospitals_in_private_sector'],
                 fill_color='YlOrRd',
                 fill_opacity=0.7,
                 line_opacity=0.8,
                 legend_name='High rate in private sector hospital',
                 highlight=True).add_to(map_choropleth_high_private)

folium.LayerControl().add_to(map_choropleth_high_private)
display(map_choropleth_high_private)

I plotted it for now, but I don't seem to know the cause of the rate. 

If I had to choose, I would say that the area with the high percentage of hospitals in public sectors seems be in the east and the area with the high percentage of hospitals in private sectors seems be in the west.

## For more data visualization

I think we can find more information with folloing data:

- Population in each states.

- Average income

- Age distribution